In [20]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sample/Sample_Submission.csv
/kaggle/input/train-and-test-dataset/Train_Data.csv
/kaggle/input/train-and-test-dataset/Test_Data.csv


In [21]:
train_path = "/kaggle/input/train-and-test-dataset/Train_Data.csv"
test_path = "/kaggle/input/train-and-test-dataset/Test_Data.csv"
sample_path = "/kaggle/input/sample/Sample_Submission.csv"

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)
sample_submission = pd.read_csv(sample_path)

train_df.head(), test_df.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


(      SEQN  RIAGENDR  PAQ605  BMXBMI  LBXGLU  DIQ010  LBXGLT  LBXIN age_group
 0  73564.0       2.0     2.0    35.7   110.0     2.0   150.0  14.91     Adult
 1  73568.0       2.0     2.0    20.3    89.0     2.0    80.0   3.85     Adult
 2  73576.0       1.0     2.0    23.2    89.0     2.0    68.0   6.14     Adult
 3  73577.0       1.0     2.0    28.9   104.0     NaN    84.0  16.15     Adult
 4  73580.0       2.0     1.0    35.9   103.0     2.0    81.0  10.92     Adult,
       SEQN  RIAGENDR  PAQ605  BMXBMI  LBXGLU  DIQ010  LBXGLT  LBXIN
 0  77017.0       1.0     1.0    32.2    96.0     2.0   135.0  15.11
 1  75580.0       2.0     2.0    26.3   100.0     2.0   141.0  15.26
 2  73820.0       1.0     2.0    28.6   107.0     2.0   136.0   8.82
 3  80489.0       2.0     1.0    22.1    93.0     2.0   111.0  12.13
 4  82047.0       1.0     1.0    24.7    91.0     2.0   105.0   3.12)

In [22]:
train_df.isnull().sum()

SEQN         12
RIAGENDR     18
PAQ605       13
BMXBMI       18
LBXGLU       13
DIQ010       18
LBXGLT       11
LBXIN         9
age_group    14
dtype: int64

In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

train_df = train_df.drop(columns=["SEQN"])

train_df = train_df.dropna(subset=["age_group"])

num_cols = ["BMXBMI", "LBXGLU", "LBXGLT", "LBXIN"]
for col in num_cols:
    train_df[col] = train_df[col].fillna(train_df[col].median())
    test_df[col] = test_df[col].fillna(test_df[col].median())

cat_cols = ["RIAGENDR", "PAQ605", "DIQ010"]
for col in cat_cols:
    train_df[col] = train_df[col].fillna(train_df[col].mode()[0])
    test_df[col] = test_df[col].fillna(test_df[col].mode()[0])

for col in cat_cols:
    le = LabelEncoder()
    train_df[col] = le.fit_transform(train_df[col])
    test_df[col] = le.transform(test_df[col])

X_train = train_df.drop(columns=["age_group"])
y_train = train_df["age_group"].map({"Adult": 0, "Senior": 1})

X_test = test_df.drop(columns=["SEQN"])

rfc = RandomForestClassifier(
    n_estimators=1000,
    max_depth=12,
    min_samples_split=4,
    min_samples_leaf=2,
    max_features='sqrt',
    class_weight='balanced',
    bootstrap=True,
    random_state=42
)

rfc.fit(X_train, y_train)

y_pred = rfc.predict(X_test)

In [25]:
submission = sample_submission.copy()
submission['age_group'] = y_pred
submission['age_group'] = submission['age_group'].astype(int)

submission.to_csv('submission.csv', index=False)